<a href="https://colab.research.google.com/github/PortugalJip2022/Portugal2022/blob/main/%E3%83%87%E3%83%A2%E4%BD%BF%E7%94%A8_%E3%81%BE%E3%81%A8%E3%82%81%E8%B3%87%E6%96%99.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 準備

## パッケージインストール

In [ ]:
p = installed.packages()
rownames( p )

[1] "abind"         "bitops"        "car"           "carData"      
  [5] "caTools"       "checkmate"     "deldir"        "e1071"        
  [9] "effects"       "estimability"  "Formula"       "gplots"       
 [13] "gridExtra"     "gtools"        "Hmisc"         "htmlTable"    
 [17] "htmlwidgets"   "insight"       "interp"        "IRdisplay"    
 [21] "IRkernel"      "jpeg"          "latticeExtra"  "lme4"         
 [25] "maptools"      "MatrixModels"  "minqa"         "mitools"      
 [29] "nloptr"        "nortest"       "numDeriv"      "pbdZMQ"       
 [33] "pbkrtest"      "png"           "proxy"         "quantreg"     
 [37] "Rcmdr"         "RcmdrMisc"     "Rcpp"          "RcppEigen"    
 [41] "readstata13"   "relimp"        "repr"          "ROCR"         
 [45] "sandwich"      "sp"            "SparseM"       "survey"       
 [49] "tcltk2"        "viridis"       "zoo"           "askpass"      
 [53] "assertthat"    "backports"     "base64enc"     "bit"          
 [57] "bit64"         "blob"          "brew"          "brio"         
 [61] "broom"         "bslib"         "cachem"        "callr"        
 [65] "cellranger"    "cli"           "clipr"         "colorspace"   
 [69] "commonmark"    "cpp11"         "crayon"        "credentials"  
 [73] "curl"          "data.table"    "DBI"           "dbplyr"       
 [77] "desc"          "devtools"      "diffobj"       "digest"       
 [81] "dplyr"         "dtplyr"        "ellipsis"      "evaluate"     
 [85] "fansi"         "farver"        "fastmap"       "forcats"      
 [89] "fs"            "gargle"        "generics"      "gert"         
 [93] "ggplot2"       "gh"            "gitcreds"      "glue"         
 [97] "googledrive"   "googlesheets4" "gtable"        "haven"        
[101] "highr"         "hms"           "htmltools"     "httr"         
[105] "ids"           "ini"           "isoband"       "jquerylib"    
[109] "jsonlite"      "knitr"         "labeling"      "lifecycle"    
[113] "lubridate"     "magrittr"      "memoise"       "mime"         
[117] "modelr"        "munsell"       "openssl"       "pillar"       
[121] "pkgbuild"      "pkgconfig"     "pkgload"       "praise"       
[125] "prettyunits"   "processx"      "progress"      "ps"           
[129] "purrr"         "R6"            "rappdirs"      "rcmdcheck"    
[133] "RColorBrewer"  "readr"         "readxl"        "rematch"      
[137] "rematch2"      "remotes"       "reprex"        "rlang"        
[141] "rmarkdown"     "roxygen2"      "rprojroot"     "rstudioapi"   
[145] "rversions"     "rvest"         "sass"          "scales"       
[149] "selectr"       "sessioninfo"   "stringi"       "stringr"      
[153] "svglite"       "sys"           "systemfonts"   "testthat"     
[157] "tibble"        "tidyr"         "tidyselect"    "tidyverse"    
[161] "tinytex"       "tzdb"          "usethis"       "utf8"         
[165] "uuid"          "vctrs"         "viridisLite"   "vroom"        
[169] "waldo"         "whisker"       "withr"         "xfun"         
[173] "xml2"          "xopen"         "yaml"          "zip"          
[177] "base"          "boot"          "class"         "cluster"      
[181] "codetools"     "compiler"      "datasets"      "foreign"      
[185] "graphics"      "grDevices"     "grid"          "KernSmooth"   
[189] "lattice"       "MASS"          "Matrix"        "methods"      
[193] "mgcv"          "nlme"          "nnet"          "parallel"     
[197] "rpart"         "spatial"       "splines"       "stats"        
[201] "stats4"        "survival"      "tcltk"         "tools"        
[205] "utils"

In [ ]:
install.packages( "Rcmdr" )

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
library("Rcmdr")

In [ ]:
install.packages( "ROCR" )

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
llrt <- function(mfull,m){
  loglik_full <- sum(mfull$y * log(mfull$fitted.values) + (1 - mfull$y) * log(1-mfull$fitted.values))
  logLik_dvar <- sum(m$y * log(m$fitted.values) + (1 - m$y) * log(1-m$fitted.values))
  chisq <- -2 * (logLik_dvar  - loglik_full)
  df <- summary(mfull)$df[1] - summary(m)$df[1]
  p <- 1- pchisq(chisq,df)
  round(c(chisq , df ,p ),3)
  }

In [ ]:
library(Rcmdr)
# library(Boruta)
library(dplyr)
library(ROCR)
library(splines)
library(stringr)

In [ ]:
library(Rcmdr)
library(dplyr)
library(ROCR)
library(splines)
library(stringr)

##データ準備


In [ ]:
samp <-
  read.table("https://raw.githubusercontent.com/PortugalJip2022/Portugal2022/main/bank-full.csv",
                  header=TRUE, stringsAsFactors=TRUE, sep=";", na.strings="NA", dec=".",
             strip.white=TRUE)

In [ ]:
#頭にidをふる
samp$id <- 1:nrow(samp)

##モデル投入用変数の作成

In [ ]:
# カテゴリ変数作成

samp$C_job <- as.factor(samp$job)
samp$C_marital <- as.factor(samp$marital)
samp$C_education <- as.factor(samp$education)
samp$C_default <- as.factor(samp$default)
samp$C_housing <- as.factor(samp$housing)
samp$C_loan<- as.factor(samp$loan)
samp$C_contact <- as.factor(samp$contact)
samp$C_day <- as.factor(samp$day)
samp$C_month <- as.factor(samp$month)
samp$C_poutcome <- as.factor(samp$poutcome)
samp$C_y <- as.factor(samp$y)

#目的変数をフラグに
samp$yF <- if_else(samp$y == "yes",1,0)
samp$C_yF <- as.factor(samp$yF)


In [ ]:
# 連続値作成

##年齢
samp$age_60 <- pmin(samp$age,60)

#＃平均残高
samp$balance_5000 <- pmax(samp$balance,-1)
samp$balance_5000 <- pmin(samp$balance_5000,5000)

##現在のマーケティングキャンペーンにおける顧客への連絡回数
samp$campaign_11 <- pmin(samp$campaign,11)

##以前のマーケティングキャンペーンにおける顧客への最終連絡日からの経過日数
samp$pdays_120 <- pmin(samp$pdays,120)

##以前のマーケティングキャンペーンにおける顧客への連絡回数
samp$previous_11 <- pmin(samp$previous,11)


#平滑化
library(splines)
i <- 3

B_age <- bs(samp$age_60,df=i)
B_balance <- bs(samp$balance_5000,df = i)
B_campaign <- bs(samp$campaign_11,df = i)
B_pdays <- bs(samp$pdays_120,df = i)
B_previous <- bs(samp$previous_11,df = i)

samp[,"B_age"] <- B_age
samp[,"B_balance"] <- B_balance
samp[,"B_campaign"] <- B_campaign
samp[,"B_pdays"] <- B_pdays
samp[,"B_previous"] <- B_previous


上記データをインサンプルデータとアウトサンプルデータに分割させる。

In [ ]:
# インサンプル作成
set.seed(3)
samp_i <- samp %>% sample_frac(0.5)

# アウトサンプル作成
samp_o <- samp[!(samp$id %in% samp_i$id), ]

# デモ（実行部分）  
ロジスティック回帰モデル実行、説明力算出

In [ ]:
#ステップワイズ最終結果 モデル作成

mbs2 <- glm(yF  ~
            B_previous + C_poutcome + C_month + C_day + C_contact +
    C_loan + C_housing + C_education + C_marital + B_pdays +
    B_campaign + B_balance + B_age + B_previous:C_contact + C_poutcome:B_pdays +
    C_month:B_pdays + C_loan:B_pdays + C_housing:B_pdays + C_month:B_campaign +
    C_contact:B_campaign + C_housing:B_campaign + C_education:B_campaign +
    C_poutcome:B_balance + C_poutcome:B_age + C_contact:B_age
            , data=samp_i,family=binomial(link=logit))

Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”


In [ ]:
mbs2


Call:  glm(formula = yF ~ B_previous + C_poutcome + C_month + C_day + 
    C_contact + C_loan + C_housing + C_education + C_marital + 
    B_pdays + B_campaign + B_balance + B_age + B_previous:C_contact + 
    C_poutcome:B_pdays + C_month:B_pdays + C_loan:B_pdays + C_housing:B_pdays + 
    C_month:B_campaign + C_contact:B_campaign + C_housing:B_campaign + 
    C_education:B_campaign + C_poutcome:B_balance + C_poutcome:B_age + 
    C_contact:B_age, family = binomial(link = logit), data = samp_i)

Coefficients:
                     (Intercept)                       B_previous1  
                       6.857e-02                         2.214e-01  
                     B_previous2                       B_previous3  
                       5.882e-01                         5.525e-01  
                 C_poutcomeother                 C_poutcomesuccess  
                      -4.174e+00                         2.423e+00  
               C_poutcomeunknown                        C_monthaug  
 

mbs2 記入して実行（中身見せる）

In [ ]:
# アウトサンプル 説明力算出

df_o_out <- data.frame(samp_o,p=predict(mbs2,newdata = samp_o,type="response"))

d_arcap <- df_o_out

#目的変数名
targetName <- "yF"
targetPred <- d_arcap[,targetName]

#ソートする変数、予測確率、GNO、CR
sortName <- "p"
sortPred <- d_arcap[,sortName]

#ARの計算
pred <- prediction(sortPred,targetPred)
AUC <- as.numeric( performance(pred,"auc")@y.values )
AR <- 2 *(AUC-0.5) * 100 #AR値

#CAP値の計算
#予測確率（GNO、CR）を照準に件数等分１０分類
d_arcap$CP10 <- bin.var( sortPred, bins=10, method='proportions',labels =FALSE)

#区分別のデフォルト件数
dcount <- tapply(targetPred,list(CP10=d_arcap$CP10),sum,na.rm=TRUE)

#CAP20値
CAP20 <- sum(dcount[9:10]) / sum(dcount) * 100

#CAP30値
CAP30 <- sum(dcount[8:10]) / sum(dcount) * 100

#CAP70値
CAP70 <- sum(dcount[4:10]) / sum(dcount) * 100

#アウトサンプル説明力結果
round(c(AR,CAP20,CAP30,CAP70),2)

Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
“prediction from a rank-deficient fit may be misleading”


[1] 55.41 58.10 68.11 89.58

# 表示用資料

##各変数の効き  

In [ ]:
#モデル作成_ステップワイズ結果版（ロジスティック回帰）

mbs <- glm(yF  ~
            B_previous + C_poutcome + C_month + C_day + C_contact +
    C_loan + C_housing + C_education + C_marital + B_pdays +
    B_campaign + B_balance + B_age + B_previous:C_contact + C_poutcome:B_pdays +
    C_month:B_pdays + C_loan:B_pdays + C_housing:B_pdays + C_month:B_campaign +
    C_contact:B_campaign + C_housing:B_campaign + C_education:B_campaign +
    C_poutcome:B_balance + C_poutcome:B_age + C_contact:B_age
            , data=samp_i,family=binomial(link=logit))

Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”


In [ ]:
Anova(mbs)


Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”
Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”
Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”
Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”
Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”
Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”
Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”
Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”
Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”
Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”
Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”
Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”
Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”

,LR Chisq,Df,Pr(>Chisq)
,<dbl>,<dbl>,<dbl>
B_previous,4.773367,3,1.891647e-01
C_poutcome,314.751080,3,6.382171e-68
C_month,357.893400,11,5.174079e-70
C_day,89.238950,30,8.561608e-08
C_contact,241.749787,2,3.196691e-53
C_loan,25.502347,1,4.418451e-07
C_housing,79.126940,1,5.824464e-19
C_education,12.886837,3,4.887829e-03
C_marital,35.647564,2,1.816477e-08


##作業先まとめ
[ロジスティック回帰の流れ](https://colab.research.google.com/drive/1TxruPYC9N0TIrfII38BA7TLLMWEj0ZT1#scrollTo=4SNmOA7PAiKH)    
[試した機会学習手法](https://colab.research.google.com/drive/1-_nX4H7RSJvf8SK8QFQKjUtdd61MFBIG#scrollTo=Av5K4D2_7fUs)